<a href="https://colab.research.google.com/github/B10956048/113_Finance/blob/main/week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import numpy as np

In [8]:
# 股票列表和時間範圍
stock_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'JPM', 'V']
start_date = '2020-01-01'
end_date = '2023-01-01'

# 獲取歷史數據
def fetch_stock_data(symbols, start, end):
    data = yf.download(symbols, start=start, end=end)['Adj Close']
    return data

# 計算每日回報率、年化回報率和年化風險
def calculate_returns_and_risk(data):
    daily_returns = data.pct_change().dropna()
    annual_returns = daily_returns.mean() * 252  # 年化回報率
    annual_risks = daily_returns.std() * np.sqrt(252)  # 年化風險
    correlation_matrix = daily_returns.corr()  # 相關性矩陣
    return annual_returns, annual_risks, correlation_matrix

# 適應函數
def fitness_function(portfolio, annual_returns, annual_risks, correlation_matrix):
    if sum(portfolio) == 0:  # 確保至少選擇一個資產
        return -np.inf
    portfolio_return = np.dot(portfolio, annual_returns)
    covariance_matrix = np.outer(annual_risks, annual_risks) * correlation_matrix
    portfolio_variance = np.dot(portfolio, np.dot(covariance_matrix, portfolio))
    portfolio_risk = np.sqrt(portfolio_variance)
    if portfolio_risk == 0:
        return -np.inf
    sharpe_ratio = portfolio_return / portfolio_risk
    return sharpe_ratio

# 基因演算法的實現
def genetic_algorithm(data, pop_size, num_generations, mutation_rate):
    annual_returns, annual_risks, correlation_matrix = calculate_returns_and_risk(data)
    num_assets = len(data.columns)

    # 初始種群生成（權重化）
    population = np.random.dirichlet(np.ones(num_assets), pop_size)  # 每個個體權重總和為 1
    best_solution = None
    best_fitness = -np.inf

    for generation in range(num_generations):
        fitness_scores = np.array([
            fitness_function(ind, annual_returns, annual_risks, correlation_matrix)
            for ind in population
        ])
        new_population = []

        # 選擇過程（輪盤選擇）
        probabilities = (fitness_scores - fitness_scores.min()) + 1e-6  # 避免負數或零
        probabilities /= probabilities.sum()
        for _ in range(pop_size // 2):
            parent1 = population[np.random.choice(len(population), p=probabilities)]
            parent2 = population[np.random.choice(len(population), p=probabilities)]

            # 單點交叉
            crossover_point = np.random.randint(1, num_assets - 1)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

            # 突變
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # 確保權重總和為 1
            child1 /= child1.sum()
            child2 /= child2.sum()
            new_population.extend([child1, child2])

        population = np.array(new_population)
        current_best = population[np.argmax(fitness_scores)]
        current_best_fitness = max(fitness_scores)

        if current_best_fitness > best_fitness:
            best_solution = current_best
            best_fitness = current_best_fitness

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.4f}")

    return best_solution, best_fitness, annual_returns, annual_risks, correlation_matrix


# 突變函數
def mutate(child, mutation_rate):
    for i in range(len(child)):
        if np.random.rand() < mutation_rate:
            child[i] += np.random.normal(0, 0.1)  # 添加小的隨機波動
    child = np.maximum(0, child)  # 確保權重非負
    return child

In [10]:
# 執行程式
stock_data = fetch_stock_data(stock_symbols, start_date, end_date)
print(stock_data)
best_portfolio, best_fitness, annual_returns, annual_risks, correlation_matrix = genetic_algorithm(
    stock_data, pop_size=100, num_generations=100, mutation_rate=0.1
)

# 結果輸出
selected_assets = [stock_symbols[i] for i in range(len(best_portfolio)) if best_portfolio[i] > 0.01]
portfolio_return = np.dot(best_portfolio, annual_returns)
cov_matrix = np.outer(annual_risks, annual_risks) * correlation_matrix
portfolio_var = np.dot(best_portfolio, np.dot(cov_matrix, best_portfolio))
portfolio_risk = np.sqrt(portfolio_var)

print("\n最佳投資組合:")
print(f"選擇的股票: {selected_assets}")
print(f"權重: {best_portfolio}")
print(f"回報率: {portfolio_return:.4f}")
print(f"風險: {portfolio_risk:.4f}")
print(f"夏普比率: {best_fitness:.4f}")

[*********************100%***********************]  10 of 10 completed


Ticker            AAPL       AMZN      GOOGL         JPM        META  \
Date                                                                   
2020-01-02   72.796021  94.900497  68.186813  122.104614  209.150269   
2020-01-03   72.088310  93.748497  67.830116  120.493263  208.043610   
2020-01-06   72.662704  95.143997  69.638054  120.397438  211.961807   
2020-01-07   72.320976  95.343002  69.503548  118.350616  212.420410   
2020-01-08   73.484352  94.598503  69.998253  119.273903  214.573944   
...                ...        ...        ...         ...         ...   
2022-12-23  130.487808  85.250000  88.907700  124.506897  117.685661   
2022-12-27  128.676834  83.040001  87.074348  124.943161  116.529144   
2022-12-28  124.728355  81.820000  85.709297  125.626007  115.272926   
2022-12-29  128.261200  84.180000  88.130516  126.346802  119.899002   
2022-12-30  128.577866  84.000000  87.911316  127.181366  119.978760   

Ticker            MSFT        NFLX       NVDA        TSLA      